In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
import google.genai as genai
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

In [7]:
loader = PyPDFLoader("data/textbook.pdf")
documents = loader.load()


In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
texts = text_splitter.split_documents(documents)

In [10]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004"
)

In [11]:
vectorstore = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory="chroma_db"
)

In [12]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

In [13]:
prompt = ChatPromptTemplate.from_template(
    """
    You are a helpful study assistant.
    Use ONLY the context below to answer the question.

    Context:
    {context}

    Question:
    {question}

    Answer:
    """
)


In [30]:
from google import genai

client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

print("Available models:")
for model in client.models.list():
    print(f"  {model.name}")


Available models:
  models/embedding-gecko-001
  models/gemini-2.5-flash
  models/gemini-2.5-pro
  models/gemini-2.0-flash-exp
  models/gemini-2.0-flash
  models/gemini-2.0-flash-001
  models/gemini-2.0-flash-exp-image-generation
  models/gemini-2.0-flash-lite-001
  models/gemini-2.0-flash-lite
  models/gemini-2.0-flash-lite-preview-02-05
  models/gemini-2.0-flash-lite-preview
  models/gemini-exp-1206
  models/gemini-2.5-flash-preview-tts
  models/gemini-2.5-pro-preview-tts
  models/gemma-3-1b-it
  models/gemma-3-4b-it
  models/gemma-3-12b-it
  models/gemma-3-27b-it
  models/gemma-3n-e4b-it
  models/gemma-3n-e2b-it
  models/gemini-flash-latest
  models/gemini-flash-lite-latest
  models/gemini-pro-latest
  models/gemini-2.5-flash-lite
  models/gemini-2.5-flash-image
  models/gemini-2.5-flash-preview-09-2025
  models/gemini-2.5-flash-lite-preview-09-2025
  models/gemini-3-pro-preview
  models/gemini-3-flash-preview
  models/gemini-3-pro-image-preview
  models/nano-banana-pro-preview
  mo

In [31]:
llm = ChatGoogleGenerativeAI(
    model="  models/embedding-gecko-001",
    temperature=0.2
)

In [32]:
rag_chain = (
    {"context": retriever,
     "question": RunnablePassthrough()
    }
    | prompt | llm
)

In [33]:
query = "What is the general idea of The Evolution of Natural Philosophy into Modern Physics"
response = rag_chain.invoke(query)
print(response.content)


ChatGoogleGenerativeAIError: Error calling model '  models/embedding-gecko-001' (Not Found): 404 Not Found. {'message': '', 'status': 'Not Found'}